In [1]:
#import libraries
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import requests
import json
import datetime 
import sqlalchemy
import random
import numpy as np

In [4]:
#create function
def get_population(cities):
           
    #create the dataframe
    headers = ["city_id", "city_name", "population"]
    population = pd.DataFrame(columns=headers)
    
    
    #create a for loop to estract the required information for each city
    for city_url in cities:
        r = requests.get(city_url)
        soup = bs(r.content, 'html.parser')
        city_name = soup.find("span", class_="mw-page-title-main").string
        pop = soup.select_one('th.infobox-header:-soup-contains("Population")').parent.find_next_sibling().find(text=re.compile(r'\d+'))
    
        #append the estracted data to the dataframe
        population = population.append({
            "city_name": city_name,
            "population": pop
        }, ignore_index=True)
        
        
       #create a unique id for each city  
        conditions = [
            (population['city_name'] == 'Berlin'),
            (population['city_name'] == 'Hamburg'),
            (population['city_name'] == 'London'),
            (population['city_name'] == 'Manchester'),
            (population['city_name'] == 'Barcelona')
        ]
        values = [1, 2, 3,4,5]
        population['city_id'] = np.select(conditions, values, default=None)
        
    #drop the city_name column    
    population = population.drop(columns=['city_name'])
    
    return population

    #urls for each city wikipedia
cities = [
    "https://en.wikipedia.org/wiki/Berlin",
    "https://en.wikipedia.org/wiki/Hamburg",
    "https://en.wikipedia.org/wiki/London",
    "https://en.wikipedia.org/wiki/Manchester",
    "https://en.wikipedia.org/wiki/Barcelona"
]







In [5]:
#assign the function to the city_info dataframe
population = get_population(cities)
population

C:\Users\JodieHorrocks\AppData\Local\Temp\ipykernel_9232\171591471.py:14: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  pop = soup.select_one('th.infobox-header:-soup-contains("Population")').parent.find_next_sibling().find(text=re.compile(r'\d+'))
C:\Users\JodieHorrocks\AppData\Local\Temp\ipykernel_9232\171591471.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  population = population.append({
C:\Users\JodieHorrocks\AppData\Local\Temp\ipykernel_9232\171591471.py:14: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  pop = soup.select_one('th.infobox-header:-soup-contains("Population")').parent.find_next_sibling().find(text=re.compile(r'\d+'))
C:\Users\JodieHorrocks\AppData\Local\Temp\ipykernel_9232\171591471.py:17: FutureWarning: The frame.append method is deprecated and will be removed f

,city_id,population
0,1,"3,850,809"
1,2,"1,945,532"
2,3,"8,799,800"
3,4,"551,938"
4,5,"1,620,343"


In [6]:
#create connection to MySQL
schema="gans"
host="127.0.0.1"
user="root"
password="LexiHarley0323"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [7]:
#create the population table in MySQL
population.to_sql('population', 
              if_exists='append', 
              con=con, 
              index=False)

5